In [1]:
# Damage Cost Estimation Script for RescueNet (using .npy Ground Truth)
# Includes Pixel Count Validation Checks
# ChatGPT and gemini were usedfor structuring and and coding
#!/usr/bin/env python3

import os
import time
import numpy as np

# CONFIGURATION SECTION

# Dataset class labels
CLASS_LABELS = {
    'Background': 0,
    'Water': 1,
    'Building_No_Damage': 2,
    'Building_Minor_Damage': 3,
    'Building_Major_Damage': 4,
    'Building_Total_Destruction': 5,
    'Vehicle': 6,
    'Road-Clear': 7,
    'Road-Blocked': 8,
    'Tree': 9,
    'Pool': 10
}

# Cost factors ($ per square meter)
COST_FACTORS_PER_SQ_M = {
    CLASS_LABELS['Background']: 0.0,
    CLASS_LABELS['Water']: 0.0,
    CLASS_LABELS['Building_No_Damage']: 0.0,
    CLASS_LABELS['Building_Minor_Damage']: 50.0,
    CLASS_LABELS['Building_Major_Damage']: 300.0,
    CLASS_LABELS['Building_Total_Destruction']: 1200.0,
    CLASS_LABELS['Vehicle']: 0.0,
    CLASS_LABELS['Road-Clear']: 0.0,
    CLASS_LABELS['Road-Blocked']: 20.0,
    CLASS_LABELS['Tree']: 0.0,
    CLASS_LABELS['Pool']: 0.0,
}

# Physical area assumption
GSD_METERS_PER_PIXEL = 0.1   # meters per pixel
AREA_PER_PIXEL_SQ_M = GSD_METERS_PER_PIXEL ** 2

# Directory containing your .npy label and prediction files
DATA_DIR = 'data/predictions/'


# FUNCTION: calculate damage cost with validation
def calculate_damage_cost_with_validation(npy_path, class_labels, cost_factors, area_per_pixel):
    """
    Load a .npy mask, count pixels per class, compute cost,
    and validate counts against np.unique.
    Returns: (total_cost, pixel_counts_dict)
    """
    try:
        mask = np.load(npy_path)
    except Exception as e:
        print(f"[WARN] could not load {npy_path}: {e}")
        return 0.0, {}

    total_cost = 0.0
    pixel_counts = {}
    counted = 0
    h, w = mask.shape
    total_pixels = h * w

    # Count & cost by class
    for name, val in class_labels.items():
        cnt = int((mask == val).sum())
        pixel_counts[name] = cnt
        counted += cnt
        cost_per_sqm = cost_factors.get(val, 0.0)
        if cost_per_sqm > 0:
            total_cost += cnt * area_per_pixel * cost_per_sqm

    # Validation 1: total pixel sum
    if counted != total_pixels:
        print(f"\n!!! WARNING: pixel-sum mismatch in {os.path.basename(npy_path)}")
        print(f"    image pixels={total_pixels}, counted={counted}\n")

    # Validation 2: np.unique check
    uniq_vals, uniq_counts = np.unique(mask, return_counts=True)
    uniq_dict = dict(zip(uniq_vals, uniq_counts))
    for name, val in class_labels.items():
        loop_cnt = pixel_counts[name]
        uniq_cnt = uniq_dict.get(val, 0)
        if loop_cnt != uniq_cnt and (loop_cnt or uniq_cnt):
            print(f"\n!!! WARNING: np.unique mismatch for '{name}' in {os.path.basename(npy_path)}")
            print(f"    loop={loop_cnt}, unique={uniq_cnt}\n")

    return total_cost, pixel_counts


# FUNCTION: pixel-wise accuracy
def pixel_accuracy(pred_mask, gt_mask):
    """
    Compute fraction of pixels where pred == gt.
    Raises ValueError on shape mismatch.
    """
    if pred_mask.shape != gt_mask.shape:
        raise ValueError(f"Shape mismatch: pred {pred_mask.shape} vs gt {gt_mask.shape}")
    return np.mean(pred_mask == gt_mask)

# MAIN FUNCTION
if __name__ == "__main__":
    print("\n" + "="*60)
    print("Starting Damage Cost, Segmentation Accuracy & Cost Error Evaluation")
    print("="*60 + "\n")

    # Print configuration
    print(f"Data directory    : {DATA_DIR}")
    print(f"GSD (m/pixel)     : {GSD_METERS_PER_PIXEL}")
    print(f"Area/pixel (m²)   : {AREA_PER_PIXEL_SQ_M:.4f}")
    print("Cost factors ($/m²):")
    for v in sorted(COST_FACTORS_PER_SQ_M):
        name = [k for k,val in CLASS_LABELS.items() if val==v][0]
        print(f"  {name:25s}({v}): {COST_FACTORS_PER_SQ_M[v]:7.2f}")
    print("-"*60 + "\n")

    # Gather prediction files
    all_files = sorted(os.listdir(DATA_DIR))
    pred_files = [f for f in all_files if f.endswith('_predict.npy')]

    if not pred_files:
        print("[ERROR] No '*_predict.npy' files found in:", DATA_DIR)
        exit(1)

    accuracies = []
    cost_errors = []

    start_time = time.time()

    # Loop over each prediction
    for fn in pred_files:
        pred_path  = os.path.join(DATA_DIR, fn)
        label_fn   = fn.replace('_predict.npy', '_label.npy')
        label_path = os.path.join(DATA_DIR, label_fn)

        if not os.path.isfile(label_path):
            print(f"[WARN] missing ground-truth for {fn}")
            continue

        # Compute ground-truth and predicted costs
        gt_cost, _   = calculate_damage_cost_with_validation(label_path,
                             CLASS_LABELS, COST_FACTORS_PER_SQ_M, AREA_PER_PIXEL_SQ_M)
        pred_cost, _ = calculate_damage_cost_with_validation(pred_path,
                             CLASS_LABELS, COST_FACTORS_PER_SQ_M, AREA_PER_PIXEL_SQ_M)

        # Load masks and compute segmentation accuracy
        gt_mask   = np.load(label_path)
        pred_mask = np.load(pred_path)
        try:
            acc = pixel_accuracy(pred_mask, gt_mask)
        except ValueError as e:
            print(f"[ERROR] {fn}: {e}")
            continue

        # Compute cost error
        abs_err = abs(pred_cost - gt_cost)
        pct_err = (abs_err / gt_cost * 100) if gt_cost > 0 else float('nan')

        # Store for summary
        accuracies.append(acc)
        cost_errors.append(pct_err)

        # Print per-image summary
        print(f"{fn}:")
        print(f"    Segmentation accuracy : {acc*100:6.2f}%")
        print(f"    Ground-truth cost     : ${gt_cost:,.2f}")
        print(f"    Predicted cost        : ${pred_cost:,.2f}")
        print(f"    Absolute cost error   : ${abs_err:,.2f}")
        print(f"    Percent cost error    : {pct_err:6.2f}%\n")

    elapsed = time.time() - start_time

    # Summary
    print("-"*60)
    print(f"Total processing time       : {elapsed:.2f} seconds")
    print(f"Average pixel-wise accuracy : {np.mean(accuracies)*100:.2f}%")
    print(f"Average percent cost error  : {np.nanmean(cost_errors):.2f}%")
    print("="*60 + "\n")




Starting Damage Cost, Segmentation Accuracy & Cost Error Evaluation

Data directory    : data/predictions/
GSD (m/pixel)     : 0.1
Area/pixel (m²)   : 0.0100
Cost factors ($/m²):
  Background               (0):    0.00
  Water                    (1):    0.00
  Building_No_Damage       (2):    0.00
  Building_Minor_Damage    (3):   50.00
  Building_Major_Damage    (4):  300.00
  Building_Total_Destruction(5): 1200.00
  Vehicle                  (6):    0.00
  Road-Clear               (7):    0.00
  Road-Blocked             (8):   20.00
  Tree                     (9):    0.00
  Pool                     (10):    0.00
------------------------------------------------------------

11160_predict.npy:
    Segmentation accuracy :  81.85%
    Ground-truth cost     : $779,735.50
    Predicted cost        : $425,900.50
    Absolute cost error   : $353,835.00
    Percent cost error    :  45.38%

12078_predict.npy:
    Segmentation accuracy :  83.51%
    Ground-truth cost     : $4,920,014.50
    Pre